In [115]:
import numpy as np
import pandas as pd

import pickle

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import MultinomialNB

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.preprocessing import normalize, MinMaxScaler

from sklearn.decomposition import TruncatedSVD

from sklearn.metrics import f1_score

In [2]:
python =pd.read_csv("/Users/Yash/Downloads/topublish/python/python_qid2all.txt", sep ='\t', \
                    names = ['qID', 'qHeader', 'qDescription', 'topVotedAnswer', 'type'])
python['type']= 'python'
java =pd.read_csv("/Users/Yash/Downloads/topublish/java/java_qid2all.txt", sep ='\t', \
                    names = ['qID', 'qHeader', 'qDescription', 'topVotedAnswer', 'type'])
java['type']= 'java'
javascript =pd.read_csv("/Users/Yash/Downloads/topublish/javascript/javascript_qid2all.txt", sep ='\t', \
                    names = ['qID', 'qHeader', 'qDescription', 'topVotedAnswer', 'type'])
javascript['type']= 'javascript'

In [3]:
python_score =pd.read_csv("/Users/Yash/Downloads/topublish/python/python_cosidf.txt", sep ='\t', \
                    names = ['qID_1', 'qID_2', 'score', 'label'], skiprows=1)
java_score =pd.read_csv("/Users/Yash/Downloads/topublish/java/java_cosidf.txt", sep ='\t', \
                    names = ['qID_1', 'qID_2', 'score', 'label'])
javascript_score =pd.read_csv("/Users/Yash/Downloads/topublish/javascript/javascript_cosidf.txt", sep ='\t', \
                    names = ['qID_1', 'qID_2', 'score', 'label'])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
java['qID'] = java['qID'].astype(object)
javascript['qID'] = javascript['qID'].astype(object)

In [5]:
pythonMerged = python.merge(python_score, left_on='qID', right_on='qID_1', how='inner')
javaMerged = java.merge(java_score, left_on='qID', right_on='qID_1', how='inner')
javascriptMerged = javascript.merge(javascript_score, left_on='qID', right_on='qID_1', how='inner')

In [6]:
pythonMerged.rename(columns = {'qHeader': 'qHeader_1', 'qDescription': 'qDescription_1', \
                               'topVotedAnswer': 'topVotedAnswer_1'}, inplace=True)
pythonMerged.drop(['qID'], inplace=True, axis = 1)
pythonMerged = pythonMerged.merge(python, left_on='qID_2', right_on='qID', how='inner')
pythonMerged.rename(columns = {'qHeader': 'qHeader_2', 'qDescription': 'qDescription_2', \
                               'topVotedAnswer': 'topVotedAnswer_2', 'type_x' : 'type'}, inplace=True)
pythonMerged.drop(['qID', 'type_y'], inplace=True, axis = 1)

In [7]:
javaMerged.rename(columns = {'qHeader': 'qHeader_1', 'qDescription': 'qDescription_1', \
                               'topVotedAnswer': 'topVotedAnswer_1'}, inplace=True)
javaMerged.drop(['qID'], inplace=True, axis = 1)
javaMerged = javaMerged.merge(java, left_on='qID_2', right_on='qID', how='inner')
javaMerged.rename(columns = {'qHeader': 'qHeader_2', 'qDescription': 'qDescription_2', \
                               'topVotedAnswer': 'topVotedAnswer_2', 'type_x' : 'type'}, inplace=True)
javaMerged.drop(['qID', 'type_y'], inplace=True, axis = 1)

In [8]:
javascriptMerged.rename(columns = {'qHeader': 'qHeader_1', 'qDescription': 'qDescription_1', \
                               'topVotedAnswer': 'topVotedAnswer_1'}, inplace=True)
javascriptMerged.drop(['qID'], inplace=True, axis = 1)
javascriptMerged = javascriptMerged.merge(javascript, left_on='qID_2', right_on='qID', how='inner')
javascriptMerged.rename(columns = {'qHeader': 'qHeader_2', 'qDescription': 'qDescription_2', \
                               'topVotedAnswer': 'topVotedAnswer_2', 'type_x' : 'type'}, inplace=True)
javascriptMerged.drop(['qID', 'type_y'], inplace=True, axis = 1)

In [9]:
maindf = pd.concat([pythonMerged, javaMerged, javascriptMerged], ignore_index = True)

In [10]:
pickle.dump(maindf, open('mainDf.p', 'wb'))

In [11]:
maindf.columns

Index(['qHeader_1', 'qDescription_1', 'topVotedAnswer_1', 'type', 'qID_1',
       'qID_2', 'score', 'label', 'qHeader_2', 'qDescription_2',
       'topVotedAnswer_2'],
      dtype='object')

In [12]:
maindf["concatenated"] = maindf['qHeader_1'] + maindf['qDescription_1'] + maindf['topVotedAnswer_1'] + \
                         maindf['qHeader_2'] + maindf['qDescription_2'] + maindf['topVotedAnswer_2']

In [13]:
pickle.dump(maindf, open('mainDf.p', 'wb'))

In [14]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(maindf["concatenated"].astype('str'))

In [15]:
pickle.dump(X, open('mainDfVectorizedTfidf.p', 'wb'))

In [16]:
y = np.array(maindf['label'])

In [17]:
X.shape

(425668, 47083)

## MNB on vectorized concatenated column, without score

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

### ComplementNB
https://people.csail.mit.edu/jrennie/papers/icml03-nb.pdf

In [20]:
cnb = ComplementNB()
cnb.fit(X_train, y_train.astype('int'))

ComplementNB(alpha=1.0, class_prior=None, fit_prior=True, norm=False)

In [21]:
cnb.score(X_train, y_train.astype('int'))

0.9489043674181539

In [22]:
cnb.score(X_val, y_val.astype('int'))

0.9459432725833972

In [23]:
cnb.score(X_test, y_test.astype('int'))

0.9457338846081563

In [24]:
f1_score(y_train.astype('int'), cnb.predict(X_train), average='weighted')  

0.9374687036097594

In [25]:
f1_score(y_val.astype('int'), cnb.predict(X_val), average='weighted')  

0.9356641501969951

In [26]:
f1_score(y_test.astype('int'), cnb.predict(X_test), average='weighted') 

0.93380224626867

## Multinomial NB

In [27]:
mnb = MultinomialNB()
mnb.fit(X_train, y_train.astype('int'))

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [28]:
mnb.score(X_train, y_train.astype('int'))

0.9613760320927612

In [29]:
mnb.score(X_val, y_val.astype('int'))

0.9617943074754569

In [30]:
mnb.score(X_test, y_test.astype('int'))

0.9598759631648186

In [31]:
f1_score(y_train.astype('int'), mnb.predict(X_train), average='weighted')  

0.9425252822941619

In [32]:
f1_score(y_val.astype('int'), mnb.predict(X_val), average='weighted')  

0.9431119807284577

In [33]:
f1_score(y_test.astype('int'), mnb.predict(X_test), average='weighted') 

0.9404547781211617

## MNB on vectorized concatenated column, with score¶

In [34]:
X.shape

(425668, 47083)

In [41]:
from scipy.sparse import coo_matrix, hstack, vstack

In [83]:
XScore = hstack([X, np.array(maindf['score'].astype('float64')).reshape(len(maindf), -1)])

In [85]:
X_train, X_test, y_train, y_test = train_test_split(XScore, y, test_size=0.05, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

### MultinomialNB

In [86]:
mnb = MultinomialNB()
mnb.fit(X_train, y_train.astype('int'))

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [87]:
mnb.score(X_train, y_train.astype('int'))

0.9613760320927612

In [88]:
mnb.score(X_val, y_val.astype('int'))

0.9617943074754569

In [89]:
mnb.score(X_test, y_test.astype('int'))

0.9598759631648186

In [90]:
f1_score(y_train.astype('int'), mnb.predict(X_train), average='weighted')  

0.9425252822941619

In [91]:
f1_score(y_val.astype('int'), mnb.predict(X_val), average='weighted')  

0.9431119807284577

In [92]:
f1_score(y_test.astype('int'), mnb.predict(X_test), average='weighted') 

0.9404547781211617

In [93]:
mnb.predict_log_proba(X_test)

array([[-6.98186310e-02, -2.69656060e+00],
       [-4.22424607e-03, -5.46902586e+00],
       [-1.68353918e-02, -4.09267784e+00],
       ...,
       [-7.52229140e-03, -4.89364327e+00],
       [-9.38432292e-03, -4.67340325e+00],
       [-1.38179851e-02, -4.28868530e+00]])

## MNB on vectorized concatenated column, with score, with type

In [79]:
maindf['type_code']= maindf['type'].astype('category')
maindf['type_code'] = maindf['type_code'].cat.codes

In [80]:
maindf['type_code'].value_counts()

2    192300
0    122369
1    110999
Name: type_code, dtype: int64

In [82]:
pickle.dump(maindf, open('mainDf.p', 'wb'))

In [95]:
XScoreType = hstack([XScore, np.array(maindf['type_code'].astype('int')).reshape(len(maindf), -1)])

In [96]:
XScoreType.shape

(425668, 47085)

In [97]:
X_train, X_test, y_train, y_test = train_test_split(XScoreType, y, test_size=0.05, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

### MultinomialNB

In [98]:
mnb = MultinomialNB()
mnb.fit(X_train, y_train.astype('int'))

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [99]:
mnb.score(X_train, y_train.astype('int'))

0.9613760320927612

In [100]:
mnb.score(X_val, y_val.astype('int'))

0.9617943074754569

In [101]:
mnb.score(X_test, y_test.astype('int'))

0.9598759631648186

## MNB on vectorized concatenated column, with LSA (T-SVD)

In [104]:
svd = TruncatedSVD(n_components=200, n_iter=7, random_state=42)

In [106]:
svd.fit(X)  

TruncatedSVD(algorithm='randomized', n_components=200, n_iter=7,
       random_state=42, tol=0.0)

In [116]:
Xsvd = svd.transform(X)

In [117]:
X_train, X_test, y_train, y_test = train_test_split(Xsvd, y, test_size=0.05, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [118]:
min_max_scaler = MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train)

In [119]:
mnb = MultinomialNB()
mnb.fit(X_train_minmax, y_train.astype('int'))

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [120]:
mnb.score(X_train_minmax, y_train.astype('int'))

0.9614419761227658

In [121]:
mnb.score(min_max_scaler.transform(X_val), y_val.astype('int'))

0.9618437646826083

In [122]:
mnb.score(min_max_scaler.transform(X_test), y_test.astype('int'))

0.9601108814132682